In [2]:
#from pycalphad import Model

def custom_model(model_name, contributions, basic_functions, parameter_functions, energy_functions):
    # Create the class definition as a string
    class_definition = f"class {model_name}(Model):\n"
  
    # Add contributions to the class definition
    for contribution in contributions:
        class_definition += f"    {contribution}\n"

    # Add basic functions to the class definition
    for basic_function in basic_functions:
        class_definition += f"    {basic_function}\n"
   
    # Add parameter functions to the class definition
    for method in parameter_functions:
        class_definition += f"    {method}\n"
  
    for method in energy_functions:
        class_definition += f"    {method}\n"
    # Return the class definition
    return class_definition




In [15]:
import json

def json_to_basic_functions_strings(json_input):
    function_strings = []
    template_file=open('template_functions.json')
    template_functions=json.load(template_file)
    for key in json_input:
        if key == 'none':
            break
        if key == 'default':
            for fun_name in ['__new__', '__init__', '__eq__', '__ne__', '__hash__' 'moles', 'ast', 'variables', 'degree_of_ordering', 'quantities', 'endmember_reference_model', 'get_internal_constraints', '_array_validity', '_purity_test', '_interaction_test', '_site_ratio_normalization', 'redlich_kister_sum', 'build_phase']:
                for funs in template_functions['functions']:
                    if funs['name'] == fun_name: 
                        function_strings.append(funs['content']+'\n') 
        else:
            for funs in template_functions['functions']:
                if funs['name'] == key:
                    function_strings.append(funs['content']+'\n')
    return function_strings


def json_to_parameter_functions_strings(json_input):
    function_strings = []

    for key, value in json_input.items():
        if len(value) == 2:
            function_strings.append(f"def {key}(self, dbe, {', '.join(value[1])}):")
            function_strings.append(f"\t#{value[0]}")
            function_strings.append(f'\tparam_query=(\n\t\t\t(where("phase_name") == self.phase_name) & \\\n\t\t\t(where("parameter_type") == "{value[0]}") & \\\n\t\t\t(where("constituent_array").test(self._array_validity))\n\t\t)\n\t\tparams = dbe._parameters.search(param_query)')
            function_strings.append(f"\treturn {key}\n")
        else:
            function_strings.append(f"def {key}(self, dbe):")
            function_strings.append(f"\t#{value[0]}")
            function_strings.append(f"\treturn {key}\n")
    
    return function_strings

def json_to_energy_functions_strings(json_input):
    function_strings = []
    template_file=open('template_functions.json')
    template_functions=json.load(template_file)
    for key, value in json_input.items():
        if value == 'CEF-default':
            print('Need to include redlich_kister_sum function')
            for funs in template_functions['functions']:
                if funs['name'] == key:
                    function_strings.append(funs['content']+'\n')
        else:
            function_strings.append(f"def {key}(self, dbe):")
            function_strings.append(f"\t#{value}")
            function_strings.append(f"\treturn {key}\n")
    
    return function_strings


def process_model_information(jsonfile):
    j_file=open(jsonfile)
    ModelInform=json.load(j_file)
    class_name=ModelInform['model_name']
    #contribution
    energy_contributions=ModelInform['energy_contributions']
    en_cons = [f'\t("{{key}}", "{{value}}")'.format(key=key, value=value) for key, value in energy_contributions.items()]
    energy_contributions_result_string = ["contributions = [\n{}\n\t]".format(",\n".join(en_cons))]
    #basic functions
    basic_function_input=ModelInform['basic_functions']
    basic_function_strings=json_to_basic_functions_strings(basic_function_input)
    #parameter functions
    parameter_function_input=ModelInform['parameters_functions']
    parameter_function_strings=json_to_parameter_functions_strings(parameter_function_input)
    #energy functions
    energy_function_input=ModelInform['energy_functions']
    energy_function_strings=json_to_energy_functions_strings(energy_function_input)
    return class_name, energy_contributions_result_string, basic_function_strings, parameter_function_strings, energy_function_strings

In [16]:
class_name, contributions, basic_functions, parameter_functions, energy_functions = process_model_information('Model_information.json')

# Generate the class definition
dynamic_class_code = custom_model(class_name, contributions, basic_functions, parameter_functions, energy_functions)


Need to include redlich_kister_sum function
Need to include redlich_kister_sum function


In [17]:
# Print the generated class definition
print(dynamic_class_code)

# Evaluate the generated code to create the class
#exec(dynamic_class_code)

class NewModel(Model):
    contributions = [
	("ref", "reference_energy"),
	("idmix", "ideal_mixing_energy"),
	("xsmix", "excess_mixing_energy")
	]
    
    def __new__(cls, *args, **kwargs):
        target_cls = cls._dispatch_on(*args, **kwargs)
        instance = object.__new__(target_cls)
        return instance

    
    def __init__(self, dbe, comps, phase_name, parameters=None):
        self._dbe = dbe
        self._endmember_reference_model = None
        self.components = set()
        self.constituents = []
        self.phase_name = phase_name.upper()
        phase = dbe.phases[self.phase_name]
        self.site_ratios = list(phase.sublattices)
        active_species = unpack_components(dbe, comps)
        for idx, sublattice in enumerate(phase.constituents):
            subl_comps = set(sublattice).intersection(active_species)
            self.components |= subl_comps
            # Support for variable site ratios in ionic liquid model
            if phase.model_hints.get('io

In [19]:
file_path = "model_NewModel.py"
with open(file_path, "w") as py_file:
    f=open('template_functions.json')
    temps=json.load(f)
    py_file.write(temps['imports']+'\n')
    py_file.write(dynamic_class_code)

### YAML input

In [72]:
import yaml

def yaml_to_basic_functions_strings(setting):
    function_strings = []
    template_file=open('template_functions.json')
    template_functions=json.load(template_file)
    for key in setting['model']['basic_functions']:
        if key == 'none':
            break
        if key == 'default':
            for fun_name in ['__new__', '__init__', '__eq__', '__ne__', '__hash__' 'moles', 'ast', 'variables', 'degree_of_ordering', 'quantities', 'endmember_reference_model', 'get_internal_constraints', '_array_validity', '_purity_test', '_interaction_test', '_site_ratio_normalization', 'redlich_kister_sum', 'build_phase']:
                for funs in template_functions['functions']:
                    if funs['name'] == fun_name: 
                        function_strings.append(funs['content']+'\n') 
        else:
            for funs in template_functions['functions']:
                if funs['name'] == key:
                    function_strings.append(funs['content']+'\n')
    return function_strings


def yaml_to_parameter_functions_strings(setting):
    function_strings = []
    functions=[]

    for param in setting['model']['parameters_functions']:
        if param['attributes'] is not None:
            def_string= f"def {param['parameter']}(self, dbe, {', '.join(param['attributes'])}):"
        else:
            def_string= f"def {param['parameter']}(self, dbe):"
        function_strings.append(def_string)
        if param['database_keyword'] is not None:
            keyword=param['database_keyword']
            search_string= f'\tparam_query=(\n\t\t\t(where("phase_name") == self.phase_name) & \\\n\t\t\t(where("parameter_type") == "{keyword}") & \\\n\t\t\t(where("constituent_array").test(self._array_validity))\n\t\t)\n\t\tparams = dbe._parameters.search(param_query)'
            function_strings.append(search_string)
        if param['comments'] is not None:
            comments_string=f"\t#{param['comments']}"
            function_strings.append(comments_string)
        function_strings.append(f"\treturn {param['parameter']}\n")
        functions.append(function_strings)
    
    return function_strings

def yaml_to_energy_functions_strings(setting):
    function_strings = []
    template_file=open('template_functions.json')
    template_functions=json.load(template_file)
    for ene_f in setting['model']['energy_functions']:
        if ene_f['comments'] is not None:
            comments_string=ene_f['comments']
            function_strings.append(f'#{comments_string}\n')
        if ene_f['function'] == 'CEF-default':
            print('Need to include redlich_kister_sum function')
            for funs in template_functions['functions']:
                if funs['name'] == ene_f['energy']:
                    content_string=funs['content']
                    function_strings.append(f'{content_string}\n')
        else:
            function_strings.append(f"\n    def {ene_f['energy']}(self, dbe):\n")
            function_strings.append(f"\t{ene_f['function']}\n\t\treturn {ene_f['energy']}\n")
    return function_strings


def process_model_information(yamlfile):
    with open(yamlfile, 'r') as file:
        setting=yaml.safe_load(file)
    class_name=setting['model']['name']
    #contribution
    energy_contributions=setting['model']['energy_contributions']
    en_cons = [f'\t("{{key}}", "{{value}}")'.format(key=key, value=value) for key, value in energy_contributions.items()]
    energy_contributions_result_string = ["contributions = [\n{}\n\t]".format(",\n".join(en_cons))]
    #basic functions
    basic_function_strings=yaml_to_basic_functions_strings(setting)
    #parameter functions
    parameter_function_strings=yaml_to_parameter_functions_strings(setting)
    #energy functions
    energy_function_strings=yaml_to_energy_functions_strings(setting)
    return class_name, energy_contributions_result_string, basic_function_strings, parameter_function_strings, energy_function_strings

In [73]:
class_name, contributions, basic_functions, parameter_functions, energy_functions = process_model_information('CustomModel.yaml')

# Generate the class definition
dynamic_class_code = custom_model(class_name, contributions, basic_functions, parameter_functions, energy_functions)
print(dynamic_class_code)

Need to include redlich_kister_sum function
Need to include redlich_kister_sum function
class NewModel(Model):
    contributions = [
	("ref", "reference_energy"),
	("idmix", "ideal_mixing_energy"),
	("xsmix", "excess_mixing_energy")
	]
    
    def __new__(cls, *args, **kwargs):
        target_cls = cls._dispatch_on(*args, **kwargs)
        instance = object.__new__(target_cls)
        return instance

    
    def __init__(self, dbe, comps, phase_name, parameters=None):
        self._dbe = dbe
        self._endmember_reference_model = None
        self.components = set()
        self.constituents = []
        self.phase_name = phase_name.upper()
        phase = dbe.phases[self.phase_name]
        self.site_ratios = list(phase.sublattices)
        active_species = unpack_components(dbe, comps)
        for idx, sublattice in enumerate(phase.constituents):
            subl_comps = set(sublattice).intersection(active_species)
            self.components |= subl_comps
            # Support